In [6]:
# Необходимые импорты
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import pandas as pd
import time
from tqdm.notebook import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from pathlib import Path
from typing import List

In [7]:
# сбор всей необходимой информации с карточки товара
def get_info_for_card(url: str):
        br = webdriver.Chrome()
        br.get(url)
        time.sleep(5)
        br.execute_script("window.scrollTo(0, 1200)")
        time.sleep(5)
        page_add = br.page_source
        tree = BeautifulSoup(page_add, 'html')
        
        # сбор описания
        submit = br.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/div[3]/div/div[3]/div[7]/button')
        submit.click()
        time.sleep(5)
        page_add = br.page_source
        tree_desc = BeautifulSoup(page_add, 'html')
    
        br.close()
        
        image_card = tree.find_all('div', {'class': 'slide__content img-plug'})
        image_card = image_card[0].img['src']

        try:
            image_feedbacks = tree.find_all('div', {'class': 'user-photos__item img-plug j-user-photos__item'})
            image_feedbacks = image_feedbacks[0].img['src']
        except:
            image_feedbacks = None
        
        desc = tree_desc.find_all('p', {'class': 'option__text'})
        
        return image_card, image_feedbacks, desc


# скачивание картинки
def load_image(url, path, name):
    img = requests.get(url)
    full_path = path / name
    
    with open(full_path, "wb") as out:
        out.write(img.content)

In [8]:
def parsing_items_wb(
    urls: List[str],
    folders: List[str],
    data_dir: Path,
    start_category: int = 0,
    start_item: int = 0):
#     try:
        for i in range(start_category, len(urls)):
            main_url = urls[i]
            br = webdriver.Chrome()
            br.get(main_url)
            time.sleep(5)
            page_add = br.page_source
            tree = BeautifulSoup(page_add, 'html')
            br.close()
            
            # берем только 20 первых товаров
            items = tree.find_all('a', {'class': 'product-card__link j-card-link j-open-full-product-card'})
            items_href = [items[i]['href'] for i in range(len(items))][:20]
            description = {col: [] for col in ["item", "desc"]}
            
            for j in tqdm(range(start_item, len(items_href))):
#                 print(items_href[j])
                image_card, image_feedbacks, desc = get_info_for_card(items_href[j])
                
                path_card = data_dir / folders[i] / "card"
                path_feedbacks = data_dir / folders[i] / "feedbacks"
                path_desc = data_dir / folders[i]
                
                if j==0:
                    path_card.mkdir(parents=True, exist_ok=True)
                    path_feedbacks.mkdir(parents=True, exist_ok=True)
                
                load_image(url=image_card, path=path_card, name=f"{j}.png")
                if image_feedbacks is not None:
                    load_image(url=image_feedbacks, path=path_feedbacks, name=f"{j}_feedbacks.png")
                
                description["item"].append(f"{j}.png")
                description["desc"].append(None if len(desc)==0 else desc[0].text)
            
            pd.DataFrame(description).to_csv(path_desc / "description.csv", index=False)
            print(f'Прошла категория "{Path(main_url).name}"')
                
#     except Exception as e:
#         print(e)        

In [9]:
categories = pd.read_csv("wb_categories.csv")
#categories = categories[:2]
categories

,category_1,category_2,category_3,url
0,Женщинам,Блузки и рубашки,NaN,https://www.wildberries.ru/catalog/zhenshchina...
1,Женщинам,Брюки,NaN,https://www.wildberries.ru/catalog/zhenshchina...
2,Женщинам,Верхняя одежда,NaN,https://www.wildberries.ru/catalog/zhenshchina...
3,Женщинам,"Джемперы, водолазки и кардиганы",NaN,https://www.wildberries.ru/catalog/zhenshchina...
4,Женщинам,Джинсы,NaN,https://www.wildberries.ru/catalog/zhenshchina...
...,...,...,...,...
1442,Канцтовары,Рисование и лепка,NaN,https://www.wildberries.ru/catalog/knigi-i-kan...
1443,Канцтовары,Счетный материал,NaN,https://www.wildberries.ru/catalog/knigi-i-dis...
1444,Канцтовары,Торговые принадлежности,NaN,https://www.wildberries.ru/catalog/kantstovary...
1445,Канцтовары,Чертежные принадлежности,NaN,https://www.wildberries.ru/catalog/knigi-i-kan...


In [10]:
categories_folder = categories.fillna("nan")
folders = []

for i in range(len(categories)):
    cat = list(categories_folder.iloc[i][:3])
    folder = "_".join(["&".join(i.split()) for i in cat])
    folders.append(folder)

In [11]:
data_dir = Path("E:/!sofi/Datasets/datasets/hse-mlds-project-year2/data")

In [12]:
parsing_items_wb(urls=categories.url[:2],
                 folders=folders,
                 data_dir=data_dir,
                 start_category=0,
                 start_item=0)

  0%|          | 0/20 [00:00<?, ?it/s]

Прошла категория "bluzki-i-rubashki"


  0%|          | 0/20 [00:00<?, ?it/s]

Прошла категория "bryuki-i-shorty"


# Тестирование

In [39]:
url = 'https://www.wildberries.ru/catalog/74299675/detail.aspx'
url = 'https://www.wildberries.ru/catalog/149410773/detail.aspx'

br = webdriver.Chrome()
br.get(url)
time.sleep(5)
br.execute_script("window.scrollTo(0, 1200)")
time.sleep(5)
page_add = br.page_source
tree = BeautifulSoup(page_add, 'html')

br.close()

In [13]:
#tree

In [40]:
data = tree.find_all('div', {'class': 'slide__content img-plug'})
data[0].img['src']

'https://basket-10.wbbasket.ru/vol1494/part149410/149410773/images/big/1.webp'

In [41]:
data = tree.find_all('div', {'class': 'user-photos__item img-plug j-user-photos__item'})
data[0].img['src']

'https://feedback05.wbbasket.ru/vol1744/part174414/174414618/photos/ms.webp'

___

In [26]:
url = 'https://www.wildberries.ru/catalog/zhenshchinam/odezhda/bluzki-i-rubashki'

br = webdriver.Chrome()
br.get(url)
time.sleep(5)
page_add = br.page_source
tree = BeautifulSoup(page_add, 'html')

br.close()

In [33]:
data = tree.find_all('a', {'class': 'product-card__link j-card-link j-open-full-product-card'})
print(len(data))
data[0]['href']

30


'https://www.wildberries.ru/catalog/160832044/detail.aspx'

In [39]:
items = tree.find_all('a', {'class': 'product-card__link j-card-link j-open-full-product-card'})
items_href = [items[i]['href'] for i in range(len(items))][:15]

___

In [17]:
categories = pd.read_csv("wb_categories.csv")
categories = categories[:2]
categories

,category_1,category_2,category_3,url
0,Женщинам,Блузки и рубашки,NaN,https://www.wildberries.ru/catalog/zhenshchina...
1,Женщинам,Брюки,NaN,https://www.wildberries.ru/catalog/zhenshchina...


In [24]:
Path(categories.iloc[0].url).name

'bluzki-i-rubashki'

In [25]:
categories["url"]

0    https://www.wildberries.ru/catalog/zhenshchina...
1    https://www.wildberries.ru/catalog/zhenshchina...
Name: url, dtype: object

___

In [10]:
# # скачивание картинки
# img = "https://basket-05.wbbasket.ru/vol742/part74299/74299675/images/c246x328/1.webp"

# p = requests.get(img)

# with open("data/img1.png", "wb") as out:
#     out.write(p.content)

In [ ]:
data_dir = Path("E:/!sofi/Datasets/datasets/hse-mlds-project-year2/data")

____

In [42]:
url = 'https://www.wildberries.ru/catalog/177006458/detail.aspx'

br = webdriver.Chrome()
br.get(url)
time.sleep(5)
br.execute_script("window.scrollTo(0, 1200)")
time.sleep(5)
submit = br.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/div[3]/div/div[3]/div[7]/button')
submit.click()
time.sleep(2)
page_add = br.page_source
tree = BeautifulSoup(page_add, 'html')

br.close()

In [18]:
submit = br.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/div[3]/div/div[3]/div[7]/button')
submit

<selenium.webdriver.remote.webelement.WebElement (session="390b805b4b23714e9aec1245cba2328f", element="f.4B6E31657C8F975832857013CF2DB357.d.FA372694CC5C762D8ED0CCBDB54238FB.e.78")>

In [19]:
submit.click()

In [20]:
page_add = br.page_source
tree = BeautifulSoup(page_add, 'html')

In [37]:
data = tree.find_all('p', {'class': 'option__text'})
data[0].text

'Красота, изящество, стиль и удобство\nКачественный пошив.Российское производство.\nЭлегантный вырез.Бесшовный материал.\n3-х ступенчатая застежка.\nДелает талию изящной.\n\nСтильное боди из коллекции российского бренда Delicieuse - элегантная и эффектная модель для модного женственного образа.\nБоди для танцев отличает длинный рукав, округлый вырез горловины, соблазнительно привлекающий внимание к области декольте сексуальный вырез на груди.\nВысокая посадка и открытый фасон подчеркивают изгиб бедер и делают талию изящной, эффектно выглядят в сочетании с джинсами, брюками или юбками с низкой посадкой.\n\nУ изделия трехступенчатая застежка - удлинитель с прочными крючками для удобства ношения, застегивания и расстегивания при надевании и снятии. \n\nБесшовную молодежную модель «в обтяжку» отличает качественный пошив и контроль всех этапов производства, материал премиум качества.\n\nМягко обтягивающее боди в лаконичном декоре дополнит гардероб взрослых женщин и девушек - подростков, под